In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from hdf5storage import loadmat
import h5py
from scipy import signal
import _pickle as pkl
import pywt

In [2]:
Data_dir = 'D:\Motion Data'
mat_files_dir = os.path.join(Data_dir, 'mat_files')
pkl_files_dir = os.path.join(Data_dir, 'pkl_files')
morlet_files_dir = os.path.join(Data_dir, 'morlet_files')

## Make event files

In [3]:
# patient_mat_files = os.listdir(mat_files_dir)
# patient_mat_files.remove(patient_mat_files[0])
# for patient in patient_mat_files:
#     patient_name = patient[:-10]
#     patient_events_dir = os.path.join(pkl_files_dir, patient_name)
#     os.mkdir(patient_events_dir)
#     mat_file = loadmat(os.path.join(mat_files_dir, patient))
#     eeg = mat_file['eegdat']
#     confidence = mat_file['events']['confidence']
#     motion_coherence = mat_file['events']['motionCoherence']
#     motion_direction = mat_file['events']['motionDirection']
#     detection_accuracy = mat_file['events']['accuracy']
#     events_num = eeg.shape[2]
#     for event in range(events_num):
#         event_file_name = os.path.join(patient_events_dir, 'event_{}'.format(str(event).zfill(4)))
#         event_eeg = eeg[:,:,event]
#         event_confidence = confidence[0, event]
#         event_motion_coherence = motion_coherence[0, event]
#         event_motion_direction = motion_direction[0, event]
#         event_detection_accuracy = detection_accuracy[0, event]
#         np_file = (event_eeg, event_confidence, event_motion_coherence, event_motion_direction, event_detection_accuracy)
#         np.save(event_file_name, np_file)

## Make morlet files

In [4]:
# morlet_files_dir = os.path.join(Data_dir, 'morlet_files')
# # os.mkdir(morlet_files_dir)
# patients = os.listdir(pkl_files_dir)
# patients.remove(patients[0])
# for patient in patients:
#     patient_morlet_dir = os.path.join(morlet_files_dir, patient)
#     os.mkdir(patient_morlet_dir)
#     patient_events_dir = os.path.join(pkl_files_dir, patient)
#     for event in os.listdir(patient_events_dir):
#         event_file_name = os.path.join(patient_events_dir, event)
#         npy_file_name = event_file_name + '.npy'
#         event_file = np.load(npy_file_name, allow_pickle=True)
#         event_eeg = event_file[0]
#         coef, freq = pywt.cwt(event_eeg, np.arange(1,81),'cmor0.4-1.0', sampling_period=2e-3)
#         event_morlet = np.moveaxis(abs(coef), 0, 1)
#         morlet_npy_file = (event_morlet, event_file[1], event_file[2], event_file[3], event_file[4])
#         morlet_file_name = os.path.join(patient_morlet_dir, 'morlet_{}'.format(str(event).zfill(4)))
#         np.save(morlet_file_name, morlet_npy_file)

## Train and Test split

In [5]:
np.random.seed(0)
patients = os.listdir(pkl_files_dir)
train_patients = np.random.choice(patients, int(len(patients) * 0.8), replace=False)
test_patients = np.array(list(set(patients) - set(train_patients)))

## Params

In [6]:
learning_rate = 1e-3
batch_size = 4
num_epochs = 100
total_data_size = 20983
total_train_size = 16431
total_test_size = 4552
epoch_size = total_train_size // batch_size

## Batch generator

In [7]:
# def normalize(x, min_value, max_value):
#     return (x - min_value) / (max_value - min_value)

# def normalizer_fn(x, y):
#     return normalize(x), normalize(y)

# def batch_generator(X, y, batch_size, preprocess_funcs=[]):
    
#     while True:
#         batch_patients = np.random.choice(len(X), batch_size, replace=True)
#         X_batch, y_batch = [], []
#         for patient in batch_patients:
#             patient_morlet_dir = os.path.join(morlet_files_dir, patient)
#             random_event = np.random.choice(os.listdir(patient_morlet_dir))
#             random_event_file_name = os.path.join(patient_morlet_dir, random_event)
#             random_event_file = np.load(random_event_file_name, allow_pickle=True)
#             X_batch.append(random_event_file[0])
#             y_batch.append(random_event_file[4])
#         X_batch, y_batch = np.array(X_batch), np.array(y_batch)
#         for func in preprocess_funcs:
#             X_batch, y_batch = func(X_batch, y_batch)
#         yield np.array(X_batch), np.array(y_batch)

In [8]:
# def normalize(x, min_value, max_value):
#     return (x - min_value) / (max_value - min_value)

# def normalizer_fn(x, y):
#     return normalize(x), normalize(y)

def batch_generator(X, batch_size):
    
    while True:
        batch_patient = X
        X_batch, y_batch = [], []
        patient_morlet_dir = os.path.join(morlet_files_dir, batch_patient)
        random_events = np.random.choice(os.listdir(patient_morlet_dir), batch_size, replace=True)
        for random_event in random_events:
            random_event_file_name = os.path.join(patient_morlet_dir, random_event)
            random_event_file = np.load(random_event_file_name, allow_pickle=True)
            X_batch.append(random_event_file[0][:,:,:10])
            y_batch.append(np.squeeze(random_event_file[4]))
#         X_batch, y_batch = np.array(X_batch), np.array(y_batch)
#         for func in preprocess_funcs:
#             X_batch, y_batch = func(X_batch, y_batch)
        yield np.array(X_batch), np.array(y_batch)

In [9]:
batch = batch_generator('MotionEEG_01', batch_size)
X_batch, y_batch = next(batch)

## Model

In [10]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras.backend as backend

In [11]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [12]:
tf.reset_default_graph()

In [13]:
inp = layers.Input(shape=(63, 80, 10))
label = layers.Input(shape=())

In [14]:
x = inp
for num_layer in range(1):
    filters = min(256, 2**(num_layer+6))
    x = layers.Conv2D(
        filters=filters, kernel_size=(3,3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(
        filters=filters, kernel_size=(3,3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(
        filters=filters, kernel_size=(3,3), strides=(2, 2),
        activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
pre_global_pooled = layers.Conv2D(filters=2, kernel_size=(3,3))(x)
global_pooled = layers.GlobalAveragePooling2D()(pre_global_pooled)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [15]:
logits = global_pooled
preds = layers.Softmax()(logits)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False)(
    y_true=tf.one_hot(tf.dtypes.cast(label, tf.int32), 2), y_pred=preds)
predicted_label = tf.cast(tf.argmax(preds, -1), tf.int32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.dtypes.cast(label, tf.int32), predicted_label), tf.float32))

In [16]:
trainable_params = tf.trainable_variables()
## Change 'trainable_params' for the case of transfer learning
gradient = tf.gradients(loss, trainable_params)
optimizer = tf.keras.optimizers.Adam(learning_rate)
optimizer_op = optimizer.apply_gradients(zip(gradient, trainable_params))

In [17]:
sess = tf.Session()
intializer = tf.global_variables_initializer()
sess.run(intializer)

In [19]:
print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

Default GPU Device:/device:GPU:0


In [ ]:
backend.set_learning_phase(1)
batch = batch_generator('MotionEEG_01', batch_size)
for epoch in range(100):
    print('Epoch {}'.format(epoch))   
    epoch_training_loss = []
    for iteration in range(epoch_size):
        if iteration % (epoch_size//10) == 0:
            print('\b.', end='\r')
        X_batch, y_batch = next(batch)
        _, batch_loss = sess.run([optimizer_op, loss], {inp: X_batch, label: y_batch})
        epoch_training_loss.append(batch_loss)
    print('\nEpoch training loss {}'.format(np.mean(epoch_training_loss)))